# Where do we go?

This notebook is meant to find an answer to this problem.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from inversion.individual import FatalityIndividualModel
from inversion.population.models import AnalyticalPopulationModel
from outbreak.plot import plot_outbreak
from utils.regions import OECD

%reload_ext autoreload
%autoreload 2

plt.rcParams['figure.figsize']=[32,18]
plt.rcParams['font.size']=22
plt.rcParams['font.weight']='bold'
plt.rcParams['axes.titlesize'] = 28
plt.rcParams['axes.labelsize'] = 24

plt.style.use('seaborn-whitegrid')



Bad key "text.kerning_factor" on line 4 in
/Users/rafa/opt/anaconda3/envs/coronavirus/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [43]:
from epydemic.outbreak import Outbreak, OutbreakTimeWindow, Epidemic
from epydemic.utils.helpers import build_coronavirus_epidemic
from epydemic.inversion.individual import FatalityIndividualModel
from epydemic.inversion.population.models import AnalyticalPopulationModel
from epydemic.utils.path import DATA_ROOTPATH

## Oxford Data

In [3]:
raw_oxford_df = pd.read_csv(DATA_ROOTPATH / "raw/oxford_data.csv", parse_dates=["Date"])

In [4]:
oxford_df = raw_oxford_df.drop(columns=["CountryCode"]).replace({"United States": "US", "Czech Republic": "Czechia"})\
         .pivot(index="Date", columns="CountryName")\
         .fillna(method="ffill")

oxford_df.columns = oxford_df.columns.set_levels(
    oxford_df.columns.levels[0].str.replace(" ", "_"), 
    level=0
)

In [22]:
economic_index = oxford_df["EconomicSupportIndex"]
government_index = oxford_df["GovernmentResponseIndex"]
stringency_index = oxford_df["StringencyIndex"]
containment_health_index = oxford_df["ContainmentHealthIndex"]

OECD_mask = set(OECD).intersection(economic_index.columns)

policies: pd.Index = oxford_df.columns.unique(0)


### Containment Policies

In [8]:
containment_policies = policies[policies.str.contains("C\d", regex=True)]

containment_flags = containment_policies[containment_policies.str.contains("Flag")]
containment_policies = containment_policies[~containment_policies.str.contains("Flag")]

containment_df = oxford_df[containment_policies].dropna(how="all", axis=1)
containment_flags_df = oxford_df[containment_flags].dropna(how="all", axis=1)

In [9]:
containment_dict = {}

for policy in containment_policies:
    containment_dict[policy] = containment_df[policy]
    

In [57]:
epidemic = build_coronavirus_epidemic(**containment_dict)

In [17]:
df = pd.read_csv(DATA_ROOTPATH / "oecd_outbreak_record.csv", index_col=[1, 2, 3])

Y = df["MTTF"]

Y


region          otw_start  otw_end
Belgium         43         50          0.436962
                           57         16.714349
                           64         13.507329
                           71         10.348846
                           78          9.748015
                                        ...    
United Kingdom  43         99          0.313585
                           106         0.130449
                           113         0.279397
                           120         0.322359
                           127         0.182358
Name: MTTF, Length: 188, dtype: float64

In [75]:
def build_policies(otw : OutbreakTimeWindow):
    for policy in containment_policies:
        outbreak_policy = otw[policy]
        
        print(outbreak_policy.shape)
        
        

In [76]:
        

for region, outbreak in epidemic.get_outbreaks(OECD).items():
    if region not in df.index.unique(level=0):
        print(region, "not in df")
        continue
        
    for otw in outbreak.expanding_windows():
        row = df.loc[(region, otw.start, otw.end)]
        individual_model = FatalityIndividualModel(parameters=row[["alpha", "beta", "eta"]])
        
        model = AnalyticalPopulationModel(otw, individual_model=individual_model)
        
        build_policies(otw)
        
    break
        



Australia not in df
Austria not in df
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(7,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(14,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(21,)
(28,)
(28,)
(28,)
(28,)
(28,)
(28,)
(28,)
(28,)
(35,)
(35,)
(35,)
(35,)
(35,)
(35,)
(35,)
(35,)
(42,)
(42,)
(42,)
(42,)
(42,)
(42,)
(42,)
(42,)
(49,)
(49,)
(49,)
(49,)
(49,)
(49,)
(49,)
(49,)
(56,)
(56,)
(56,)
(56,)
(56,)
(56,)
(56,)
(56,)
(63,)
(63,)
(63,)
(63,)
(63,)
(63,)
(63,)
(63,)
(70,)
(70,)
(70,)
(70,)
(70,)
(70,)
(70,)
(70,)
(77,)
(77,)
(77,)
(77,)
(77,)
(77,)
(77,)
(77,)
(84,)
(84,)
(84,)
(84,)
(84,)
(84,)
(84,)
(84,)


In [59]:
epidemic.outbreaks["Italy"].C1_School_closing

Dates
0 days      0.0
1 days      0.0
2 days      0.0
3 days      0.0
4 days      0.0
           ... 
128 days    2.0
129 days    2.0
130 days    2.0
131 days    2.0
132 days    2.0
Freq: D, Name: Italy, Length: 133, dtype: float64